In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model
from fastai.vision.all import *

In [ ]:
set_seed(999, reproducible=True)

In [ ]:
dataset_path = Path('../input/petfinder-pawpularity-score/')
dataset_path.ls()

In [ ]:
train_df = pd.read_csv(dataset_path/'train.csv')
train_df.head()

In [ ]:
train_df['path'] = train_df['Id'].map(lambda x:str(dataset_path/'train'/x)+'.jpg')
train_df = train_df.drop(columns=['Id'])
train_df = train_df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
train_df.head()

In [ ]:
len_df = len(train_df)

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(12,8)})
fig = plt.figure()
sns.histplot(data=train_df, x='Pawpularity', bins=50)
plt.axvline(train_df['Pawpularity'].mean(), c='red', ls='-', lw=2, label='Average')
plt.axvline(train_df['Pawpularity'].median(),c='green',ls='-',lw=2, label='Median')
plt.title('Distribution of Pawpularity Scores', fontsize=15) #, fontweight='bold'
plt.legend()
plt.show()


In [ ]:
train_df['norm_score'] = train_df['Pawpularity']/100

In [ ]:
im = Image.open(train_df['path'][3])
width, height = im.size
print(width,height)

In [ ]:
train_df[3:4]

In [ ]:
dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                               valid_pct=0.2, #80-20 train-validation random split
                               seed=999, #seed
                               fn_col='path', #filename/path is in the second column of the DataFrame
                               label_col='norm_score', #label is in the first column of the DataFrame
                               y_block=RegressionBlock, #The type of target
                               bs=32, #pass in batch size
                               num_workers=8,
                               item_tfms=Resize(224), #pass in item_tfms
                               batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) #pass in batch_tfms

In [ ]:
dls.show_batch()

In [ ]:
# Making pretrained weights work without needing to find the default filename
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'

In [ ]:
model = create_model('swin_large_patch4_window7_224', pretrained=True, num_classes=dls.c)

In [ ]:
def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

In [ ]:
# learn = Learner(dls, model, loss_func=BCEWithLogitsLossFlat(), metrics=petfinder_rmse).to_fp16()
learn = Learner(dls, model, loss_func=BCEWithLogitsLossFlat(), metrics=petfinder_rmse).to_fp16()

In [ ]:
learn.lr_find(end_lr = 5e-1)

In [ ]:
learn.fit_one_cycle(10, 1e-5, cbs=[SaveModelCallback(), EarlyStoppingCallback(monitor='petfinder_rmse', comp=np.less, patience=3)])

In [ ]:
learn.recorder.plot_loss()
learn = learn.to_fp32()
learn.save('fine-tuned')
learn.export()

In [ ]:
test_df = pd.read_csv(dataset_path/'test.csv')
test_df.head()

In [ ]:
test_df['Pawpularity'] = [1]*len(test_df)
test_df['path'] = test_df['Id'].map(lambda x:str(dataset_path/'test'/x)+'.jpg')
test_df = test_df.drop(columns=['Id'])
train_df['norm_score'] = train_df['Pawpularity']/100
test_dl = dls.test_dl(test_df)

In [ ]:
test_dl.show_batch()

In [ ]:
preds, _ = learn.tta(dl=test_dl, n=5, beta=0)

In [ ]:
sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
sample_df['Pawpularity'] = preds.float().numpy()*100
sample_df.to_csv('submission.csv',index=False)